# Advanced Spatial Analysis
# Module 09: Spatial weights and ESDA

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pysal as ps
import seaborn as sns
from scipy.stats import stats

np.random.seed(0)
print(gpd.__version__, ps.__version__)

## Load the tracts data set

In [ ]:
tracts = gpd.read_file('census/census_tracts_data.geojson')
tracts = tracts.set_index('index')
tracts.shape

In [ ]:
tracts.columns

In [ ]:
tracts.head()

In [ ]:
# calculate pop density in persons per sq km
tracts['pop_density'] = tracts['total_pop'] / (tracts['ALAND'] / 1e6)
tracts = tracts.replace([np.inf, -np.inf], np.nan)

In [ ]:
utm_ma = '+proj=utm +zone=18 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
tracts = tracts.to_crs(utm_ma)

## 1. Exploratory analysis

In [ ]:
def map_variable(df, col):
    values = df.dropna(subset=[col])
    ax = values.plot(column=col, scheme='quantiles', k=10, cmap='plasma', figsize=(10,10))
    ax.axis('off')
    plt.show()

In [ ]:
map_variable(tracts, 'pop_density')

In [ ]:
map_variable(tracts, 'med_household_income')

It looks like these two variables might be negatively correlated?

In [ ]:
# calculate the correlation coefficient and its p-value
x = tracts.dropna(subset=['pop_density', 'med_household_income'])['pop_density']
y = tracts.dropna(subset=['pop_density', 'med_household_income'])['med_household_income']
r, p = stats.pearsonr(x=x, y=y)
print('r={:.4f}, p={:.4f}'.format(r, p))

In [ ]:
# scatter plot them
fig, ax = plt.subplots()
ax.scatter(x=x, y=y, s=1)
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
plt.show()

In [ ]:
# estimate a simple linear regression model
m, b, r, p, se = stats.linregress(x=x, y=y)
print('m={:.4f}, b={:.4f}, r^2={:.4f}, p={:.4f}'.format(m, b, r ** 2, p))

In [ ]:
# plot the regression line with 95% CI
ax = sns.regplot(x, y, marker='.', scatter_kws={'s':2, 'color':'gray'})
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
plt.show()

## 2. Spatial weights matrix

Spatial weights define the spatial connections among our units of analysis (tracts).

### 2.1. Contiguity-based weights: rook contiguity

Using rook contiguity, two spatial units must share an edge of their boundaries to be considered neighbors. This isn't terribly common in practice (since queen is more useful, but it's worth understanding).

In [ ]:
# get the tract labels (GEOIDs) and pick one to work with later
labels = tracts.index.tolist()
label = labels[1000]

In [ ]:
%%time
w_rook = ps.lib.weights.Rook.from_dataframe(tracts)

### 2.2. Contiguity-based weights: queen contiguity

Using queen contiguity, two spatial units need only share a vertex (a single point) of their boundaries to be considered neighbors.

In [ ]:
%%time
w_queen = ps.lib.weights.Queen.from_dataframe(tracts, ids=labels, id_order=labels)

In [ ]:
# find the neighbors of some tract
w_queen.neighbors[label]

In [ ]:
# this is a raw contiguity matrix, so weights are all 1
w_queen.weights[label]

In [ ]:
# how many neighbors does this tract have?
w_queen.cardinalities[label]

In [ ]:
# convert cardinalites to series and describe data -- looks right-skewed
cardinalites_queen = pd.Series(w_queen.cardinalities)
cardinalites_queen.describe()

In [ ]:
ax = cardinalites_queen.hist(bins=20)

In [ ]:
# number of observations
w_queen.n

In [ ]:
# average number of neighbors
w_queen.mean_neighbors

In [ ]:
# min number of neighbors
w_queen.min_neighbors

In [ ]:
# max number of neighbors
w_queen.max_neighbors

In [ ]:
# islands (observations with no neighbors, disconnected in space)
w_queen.islands

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
tracts.plot(ax=ax, facecolor='#999999', edgecolor='w', linewidth=0.5)

# plot some tract of interest in red
tract = tracts.loc[[label]]
tract.plot(ax=ax, facecolor='r', edgecolor='w', linewidth=1)

# plot the neighbors in blue
neighbors = tracts.loc[w_queen[label]]
neighbors.plot(ax=ax, facecolor='b', edgecolor='w', linewidth=1)

# zoom to area of interest
xmin, ymin, xmax, ymax = neighbors.unary_union.bounds
ax.axis('equal')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_title('Neighbors of tract {}'.format(label))
plt.show()

### 3.3. Distance-based weights: k-nn

Using k-nearest neighbors and distance bands.

In [ ]:
# k-nearest neighbors finds the closest k tract centroids to each tract centroid
# here, k=6
w_knn = ps.lib.weights.KNN.from_dataframe(tracts, k=6)

### 3.4. Distance-based weights: distance band

Tracts are considered neighbors of some tract if they are within some threshold distance of it.

In [ ]:
x = tracts.centroid.x
y = tracts.centroid.y
coords = np.array([x, y]).T
threshold = ps.lib.weights.min_threshold_distance(coords)

In [ ]:
%%time
w_dist = ps.lib.weights.distance.DistanceBand.from_dataframe(tracts, threshold=threshold)

In [ ]:
len(w_dist.neighbors[label])

### 3.5. Standardizing weights

A spatial weights matrix with raw values (e.g. 1s and 0s for neighbor/not) is not always the best for analysis. Some sort of standardization is useful.

In [ ]:
w_queen[label]

In [ ]:
# check the current transformation of the weights matrix (O = original)
w_queen.transform

Typically, we want to apply a row-based transformation, so every row of the matrix sums up to 1.

In [ ]:
w_queen.transform = 'R'
w_queen[label]

PySAL supports the following transformations:

  - O: original, returning the object to the initial state
  - B: binary, with every neighbor having assigned a weight of 1
  - R: row-based, with all the neighbors of a given observation adding up to 1
  - V: variance stabilizing, with the sum of all the weights being constrained to the number of observations

**It can take a long time to calculate a weights matrix for a large data set.**

Once you've created yours, you might want to save it to disk to re-use in subsequent analyses.

In [ ]:
# save your matrix to disk
f = ps.lib.io.open('tracts_queen.gal', 'w')
f.write(w_queen)
f.close()

In [ ]:
# read a matrix from disk (notice its transformation)
w_queen = ps.lib.io.open('tracts_queen.gal', 'r').read()
w_queen[label]

## 4. Spatial lag

In [ ]:
col = 'med_household_income'
tracts_not_null = tracts[[col, 'geometry']].dropna()
y = tracts_not_null[col]

In [ ]:
w_queen = ps.lib.weights.Queen.from_dataframe(tracts_not_null)
w_queen.transform = 'R'

In [ ]:
# compute spatial lag
y_lag = ps.lib.weights.lag_spatial(w_queen, y)

In [ ]:
col_lag = '{}_lag'.format(col)
data_lag = pd.DataFrame(data={col:y, col_lag:y_lag}).astype(int)
data_lag.sample(10)

## 5. Moran plots

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
sns.regplot(x=col, y=col_lag, data=data_lag)
plt.show()

In [ ]:
# standardize the values
y_std = (y - y.mean()) / y.std()

In [ ]:
# compute the spatial lag of the standardized vector and save it as a series indexed like the original vector
y_std_lag = pd.Series(ps.lib.weights.lag_spatial(w_queen, y_std), index=y_std.index, name=col_lag)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
sns.regplot(ax=ax, x=y_std, y=y_std_lag)
plt.axvline(0, c='k', alpha=0.5)
plt.axhline(0, c='k', alpha=0.5)
plt.show()

## 6. Spatial autocorrelation

## 7. Spatial clustering